In [1]:
import sys
import os
from pathlib import Path
#
path_root = Path(os.path.abspath(''))
sys.path.insert(1, os.path.join(path_root))
#
path_root2 = Path(os.path.abspath('')).parent
sys.path.insert(1, os.path.join(path_root2))

import time
import talib
import sqlite3 as sqlite3
import yfinance as yf
from util.util import get_ucodes

path_sqlite = os.path.join(path_root, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')
if not os.path.exists(path_sqlite):
    path_sqlite = os.path.join(path_root2, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')

In [2]:
ucodes = get_ucodes()

for ucode in ucodes['美國ETF']:
    product = yf.Ticker(ucode)
    hist = product.history(period="max")
    hist = hist.drop(columns=['Dividends', 'Stock Splits'])
    hist.columns = ['open', 'high', 'low', 'close', 'volume']
    hist.insert(0, 'stime', hist.index.strftime('%Y-%m-%d'))
    hist.insert(1, 'code', ucode)
    hist = hist[['stime', 'code', 'close', 'high', 'low', 'open', 'volume']]

    conn = sqlite3.connect(path_sqlite)
    tb_name = 's_' + ucode.lower().replace('.', '')
    
    conn.execute("DROP TABLE %s" % tb_name)
    conn.execute("CREATE TABLE IF NOT EXISTS %s (stime TEXT PRIMARY KEY, code TEXT NOT NULL, close REAL, "
                 "high REAL, low REAL, open REAL, volume REAL)" % tb_name)
    
    stmt = "REPLACE INTO "+tb_name+" (stime, code, close, high, low, open, volume) VALUES (?, ?, ?, ?, ?, ?, ?)"
    conn.executemany(stmt, hist.values.tolist())
    conn.commit()
    conn.close()

In [13]:
# 指數
us_mapping = {
    '^IXIC': '.IXIC',
    '^DJI': '.DJI',
    '^GSPC': '.SP500',
    '^HSI': '.HSI',
    '^HSCE': '.HSCE',
    '000001.SS': '.SSEC',
    '399001.SZ': '.SZI',
}

us_mapping = {
    'XIN9.L': 'XIN9.L'
}

for k, v in us_mapping.items():
    product = yf.Ticker(k)
    print(product)
    hist = product.history(period="max")
    hist = hist.drop(columns=['Dividends', 'Stock Splits'])
    hist.columns = ['open', 'high', 'low', 'close', 'volume']
    hist.insert(0, 'stime', hist.index.strftime('%Y-%m-%d'))
    hist.insert(1, 'code', v)
    hist = hist[['stime', 'code', 'close', 'high', 'low', 'open', 'volume']]
    print(hist)
    """
    conn = sqlite3.connect(path_sqlite)
    tb_name = 's_' + v.lower().replace('.', '')
    
    # conn.execute("DROP TABLE %s" % tb_name)
    conn.execute("CREATE TABLE IF NOT EXISTS %s (stime TEXT PRIMARY KEY, code TEXT NOT NULL, close REAL, "
                 "high REAL, low REAL, open REAL, volume REAL)" % tb_name)
    
    stmt = "INSERT INTO IGNORE"+tb_name+" (stime, code, close, high, low, open, volume) VALUES (?, ?, ?, ?, ?, ?, ?)"
    conn.executemany(stmt, hist.values.tolist())
    conn.commit()
    """
    
    time.sleep(2)

yfinance.Ticker object <XIN9.L>
- XIN9.L: 1d data not available for startTime=-2208988800 and endTime=1627976355. Only 100 years worth of day granularity data are allowed to be fetched per request.


KeyError: "['Dividends' 'Stock Splits'] not found in axis"

In [ ]:
product = yf.Ticker('2828.HK')
hist = product.history(period="max")
hist = hist.drop(columns=['Dividends', 'Stock Splits'])
hist.columns = ['open', 'high', 'low', 'close', 'volume']
hist.insert(0, 'stime', hist.index.strftime('%Y-%m-%d'))
hist.insert(1, 'code', '2828.HK')
hist = hist[['stime', 'code', 'close', 'high', 'low', 'open', 'volume']]

conn = sqlite3.connect(path_sqlite)
tb_name = 's_02828'

conn.execute("DROP TABLE %s" % tb_name)
conn.execute("CREATE TABLE IF NOT EXISTS %s (stime TEXT PRIMARY KEY, code TEXT NOT NULL, close REAL, "
             "high REAL, low REAL, open REAL, volume REAL)" % tb_name)

stmt = "REPLACE INTO "+tb_name+" (stime, code, close, high, low, open, volume) VALUES (?, ?, ?, ?, ?, ?, ?)"
conn.executemany(stmt, hist.values.tolist())
conn.commit()
conn.close()